# **Deep Learning Inference Notebook**
### Authors: Simon Raviv, Adam Gavriely
##### Deep Learning Course BIU, 2021.
---
<br/>

In this project we were asked to train a Cycle GAN that can transfer style from images painted by Monet to 'regular' images using only 30 monet images.

The training notebook was submitted to the [Kaggle competition](https://www.kaggle.com/c/gan-getting-started) which use MiFID metric to evaluate the model performance.  
Our best MiFID score was : 

This is the best results we achieved in this project timeframe, but there's a lot more that can be done to improve the time and performance of the model.


# Load required libraries

In [ ]:
!pip install -q tensorflow-addons
!pip install -q gdown

In [ ]:
import gdown
import os
import shutil
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from glob import glob

from tensorflow_addons.layers import InstanceNormalization

# Global Parameters & Paths

In [ ]:
input_image_size = (256, 256, 3)

chosen_monet_file_id = "1diLgk2a-snDTHI7_nbqjJjKEZR8kXjPQ"
photos_zip_file_id = "1HCYFybMEle_KQaL2KBKoOPjEVq2eA0dz"
monet_generator_file_id = '1-HLVUS-OHe8qqM4pbymkdtlBUcULYxB_' #TODO CHANGE TO GOOD MODEL

chosen_monet_filename = 'chosen_monet.tfrec'
photos_tfrec_folder_name = 'photo_tfrec'
monet_generator_filename = 'monet_generator_model.h5'

# Read the data


## Download remote data

In [ ]:
def local_download_files_from_drive(id, filename, isZip=False):
    file_url = f"https://drive.google.com/uc?id={id}"
    ext = '.zip' if isZip else ''
    gdown.download(file_url, filename+ext, quiet=True)
    if isZip is True:
        shutil.unpack_archive(filename+ext, extract_dir=filename)
        os.remove(filename+ext)

In [ ]:
# Download all remote files to local memory:
local_download_files_from_drive(monet_generator_file_id, monet_generator_filename)
local_download_files_from_drive(chosen_monet_file_id, chosen_monet_filename)
local_download_files_from_drive(photos_zip_file_id, photos_tfrec_folder_name, isZip=True)

## Utility functions for downloading, reading, parsing and visualizing the images

In [ ]:
# Map values in the range [-1, 1]
def normalize_img(img):
    img = tf.cast(img, dtype=tf.float32)
    return (img / 127.5) - 1.0

# Map values in the range [0, 255]
def denormalize_img(img):
    img = tf.cast(img, dtype=tf.float32)
    return tf.cast((img + 1.0) * 127.5, tf.uint8)
    
def decode_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = normalize_img(image)
    image = tf.reshape(image, [*input_image_size])
    return image

def read_tfrecord(example):
    tfrecord_format = {
        "image_name": tf.io.FixedLenFeature([], tf.string),
        "image": tf.io.FixedLenFeature([], tf.string),
        "target": tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example['image'])
    return image

def load_dataset(filenames):
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.map(read_tfrecord)
    return dataset

def visualize_dataset(dataset, rows, cols, images_normalized=True, imsize=3):
    plt.figure(figsize=(cols*imsize,rows*imsize))
    for i, im in enumerate(dataset.take(rows*cols)):
        if images_normalized is True:
            im = denormalize_img(im)
        plt.subplot(rows, cols, i+1)
        plt.imshow(im)
        plt.axis(False)
    plt.tight_layout()
    plt.show()

## Read the NON-Monet photos

In [ ]:
filenames = glob(f"./{photos_tfrec_folder_name}/*.tfrec")
photo_dataset = load_dataset(filenames)
visualize_dataset(photo_dataset, rows=2, cols=6, images_normalized=True)

## Chosen Monet Files

In [ ]:
chosen_monet_dataset = load_dataset(chosen_monet_filename)
visualize_dataset(chosen_monet_dataset, rows=5, cols=6, images_normalized=True)

# Load the model and visualize predictions

We must define the special reflection padding layer in order to load the model

In [ ]:
class ReflectionPadding2D(tf.keras.layers.Layer):

    def __init__(self, padding=(1, 1), **kwargs):
        self.padding = tuple(padding)
        super(ReflectionPadding2D, self).__init__(**kwargs)

    def call(self, input_tensor, mask=None):
        padding_width, padding_height = self.padding
        padding_tensor = [
            [0, 0],
            [padding_height, padding_height],
            [padding_width, padding_width],
            [0, 0],
        ]
        return tf.pad(input_tensor, padding_tensor, mode="REFLECT")

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'padding': self.padding
        })
        return config

In [ ]:
monet_generator = tf.keras.models.load_model('./' + monet_generator_filename, compile=False, custom_objects={'ReflectionPadding2D': ReflectionPadding2D})

# Visualize predictions

In [ ]:
def visualize_predictions(model, images_dateset, rows, cols, imsize=3):
    plt.figure(figsize=(cols*imsize, rows*imsize))
    amount_to_show = rows*cols
    for i, img in zip(range(1, amount_to_show, 2), images_dateset.take(amount_to_show).batch(1)):
        input_img = denormalize_img(img[0]).numpy()
        plt.subplot(rows, cols, i)
        plt.title("Input image")
        plt.imshow(input_img)

        prediction = model(img, training=False).numpy()
        prediction = denormalize_img(prediction[0])
        plt.subplot(rows, cols, i+1)
        plt.title("Translated image")
        plt.imshow(prediction)

    plt.tight_layout()
    plt.show()

visualize_predictions(monet_generator, photo_dataset, rows=3, cols=6, imsize=4)

As you can see we perform pretty well on nature images such as water, skies, mountains etc. and perform not quite as well on other stuff images e.g geometric objects, animals or people.

We believe this is due to the fact that Monet chose to focus the nature as his inpiration the vast majority of his paintings and you can see from our manually selected 30 monet images that we aimed to focus on that part of his paintings to make the learning of the model more focused.


# Generate all the photos to monet images

Used for kaggle submissions

In [ ]:
from tqdm import tqdm
from PIL import Image
!mkdir ../images
i = 1
for img in tqdm(photo_dataset.batch(1)):
    prediction = monet_generator.predict(img)
    prediction = denormalize_img(prediction)
    im = Image.fromarray(prediction[0].numpy())
    im.save("../images/" + str(i) + ".jpg")
    i += 1

shutil.make_archive("/kaggle/working/images", 'zip', "/kaggle/images")